# 第五章 神经网络

## 5.1 试述将线性函数 f(x) 用作神经元激活函数的缺陷。

如果激活函数是一个线性函数，那么神经网络模型本质上是一个多元线性回归模型。

一个多元线性回归模型只能在样本点之间画出一道平面，例如二维数据(x1,x2)，一元线性回归就是画一条直线分开他们，如果实际分类函数是非线性的，那么显然分类效果是不好的。书上99—100提到过。

## 5.2 试述使用图 5.2(b) 激活函数的神经元与对率回归的联系。

隐层的输出结果就是对训练集作对率回归的结果，最终的输出结果就是对上一隐层的输出作对率回归的结果。

单隐层神经网络中，若取隐层神经元数量为1，输出层权值为1，其相当于对率回归。

PS：有点IVLS的感觉

## 5.3 对于图 5.7 中的 Vih 试推导出 BP 算法中的更新公式(5.13)。

$$
\frac{\partial E_k}{\partial v_{ih}} 
= \frac{\partial E_k}{\partial b_h} \cdot \frac{\partial b_h}{\partial a_h} \cdot \frac{\partial a_h}{\partial v_{ih}} \\
$$
显然，
$$
\frac{\partial a_h}{\partial v_{ih}} = x_i
$$
由于第$h$个隐层神经元连接着$l$个输出层神经元，故
$$
\frac{\partial E_k}{\partial b_h} \\
= \frac{\partial \frac{1}{2}\sum_{j=1}^{l}(\hat{y}_j^k - y_j^k)^2}{\partial b_h} \\
= \sum_{j=1}^{l}\frac{\partial \frac{1}{2}(\hat{y}_j^k - y_j^k)^2}{\partial b_h} \\
= \sum_{j=1}^{l}\frac{\partial E_k}{\partial \beta_j} \cdot \frac{\partial\beta_j}{\partial b_h} 
$$
根据前面算得，
$$
\frac{\partial E_k}{\partial \beta_j} = \hat{y}_j^k(1 - \hat{y}_j^k)(y_j^k - \hat{y}_j^k) = g_j
$$
又
$$
\frac{\partial b_h}{\partial a_h} = \frac{\partial f(a_h - \gamma_h)}{\partial a_h} = f(a_h - \gamma_h)(1 - f(a_h - \gamma_h)) = b_h(1 - b_h)
$$
故
$$
\frac{\partial E_k}{\partial v_{ih}} = \sum_{j=1}^{l}g_j w_{hj} b_h(1 - b_h)
$$

## 5.4 试述式(5.6) 中学习率的取值对神经网络训练的影响。

若使用梯度下降法，取值过大收敛速度较慢，取值过小有可能在极值附近反复迭代。

## 5.5 试编程实现标准 BP 算法和累积 BP 算法，在西瓜数据集 3.0 上分别用这两个算法训练一个单隐层网络，并进行比较。

一个二分类问题，样本17个，单隐层网络，故输入层神经元17个，隐层神经元数量待定，输出层神经元2个。

暂时做到这里，已实现基本标准BP和ABP算法，存在缺陷：（1）缺少自动确定隐层神经元个数部分（2）目标函数缺少正则项

In [1]:
import pandas as pd
import numpy as np

In [2]:
read_path = 'D:\JWE\Files\Courses\数据挖掘\melon_data.csv'
melon_data = pd.read_csv(read_path)
data0 = melon_data.copy() # df.copy(deep=True)才是深拷贝，对拷贝对象的操作不会影响原对象
data0.loc[data0['好瓜'] == '是',['好瓜']] = 1 # '好瓜'
data0.loc[data0['好瓜'] == '否',['好瓜']] = 0 # '坏瓜'
character_dict = {character:attr for character,attr in zip(data0.columns.drop(['编号','好瓜']),[set(data0.loc[:,[each]].values.flatten()) for each in data0.columns.drop(['编号','好瓜'])])}
temp_dict = character_dict.copy()
del temp_dict['密度'],temp_dict['含糖率']
for character in temp_dict:
    i = 0
    for each in character_dict[character]:
        data0.loc[data0[character] == each,[character]] = i
        i += 1  
data0
# character_dict

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
0,1,1,1,1,2,2,1,0.697,0.460,1
1,2,0,1,0,2,2,1,0.774,0.376,1
2,3,0,1,1,2,2,1,0.634,0.264,1
3,4,1,1,0,2,2,1,0.608,0.318,1
4,5,2,1,1,2,2,1,0.556,0.215,1
5,6,1,0,1,2,0,0,0.403,0.237,1
6,7,0,0,1,1,0,0,0.481,0.149,1
7,8,0,0,1,2,0,1,0.437,0.211,1
8,9,0,0,0,1,0,1,0.666,0.091,0
9,10,1,2,2,2,1,0,0.243,0.267,0


In [3]:
# X = np.c_[data0.drop(['编号','好瓜'],axis=1).values,np.ones(len(data0))]
X = data0.drop(['编号','好瓜'],axis=1).values
Y = data0['好瓜'].values.reshape(17,1)

sigmoid = lambda x:1 / (1 + np.exp(-x))

class BP_result(object):
    def __init__(self,predict,parameter):
        self.parameter = parameter
        self.predict = predict

def BP(X,Y,d=8,q=10,l=2,eta=0.2,max_iters=10**(3),threshold=10**(-3)):
    v_cur = np.random.rand(q,d) 
    gamma_cur = -np.random.rand(1,q) 
    w_cur = np.random.rand(l,q) 
    theta_cur = -np.random.rand(1,l) 
    while max_iters:
        max_iters -= 1
        # 参数迭代
        for x,y in zip(X,Y):
            x = x.reshape(1,len(x))
            y = np.array([[y[0],1 - y[0]]])
            alpha = np.dot(x,v_cur.T).reshape(1,q)
            b = sigmoid(alpha - gamma_cur) 
            beta = np.dot(b,w_cur.T).reshape(1,l)
            y_hat = sigmoid(beta - theta_cur) 
#             print(y_hat) # 调试
            E_k = 1/2*np.square(np.sum(y_hat - y,axis=1))
            if E_k.all() <= threshold: # E_k小于阈值，停止迭代
                parameter = v_cur,gamma_cur,w_cur,theta_cur
                alpha = np.dot(X,v_cur.T)
                b = sigmoid(alpha - gamma_cur)
                beta = np.dot(b,w_cur.T)
                predict = sigmoid(beta - theta_cur)
                return BP_result(predict=predict,parameter=parameter)
            g = y_hat*(1 - y_hat)*(y - y_hat)
#             e = b*(1 - b)*np.sum(w_cur*g.T,axis=0).reshape(1,q)
            e = b*(1 - b)*np.dot(g,w_cur).reshape(1,q)
            v_cur += eta*np.dot(e.T,x)
            gamma_cur += -eta*e
            w_cur += eta*np.dot(g.T,b)
            theta_cur += -eta*g
    parameter = v_cur,gamma_cur,w_cur,theta_cur
    alpha = np.dot(X,v_cur.T)
    b = sigmoid(alpha - gamma_cur)
    beta = np.dot(b,w_cur.T)
    predict = sigmoid(beta - theta_cur)
    return BP_result(predict=predict,parameter=parameter)

def ABP(X,Y,d=8,q=10,l=2,eta=0.2,max_iters=10**(4),threshold=10**(-3)):
    n = len(X)
    v_cur = np.random.rand(q,d) 
    gamma_cur = -np.random.rand(n,q) 
    w_cur = np.random.rand(l,q) 
    theta_cur = -np.random.rand(n,l) 
    while max_iters:
        max_iters -= 1
        # 参数迭代
        alpha = np.dot(X,v_cur.T)
        b = sigmoid(alpha - gamma_cur)
        beta = np.dot(b,w_cur.T)
        Y_hat = sigmoid(beta - theta_cur)
        E = np.sum(1/2*np.square(np.sum(Y_hat - Y,axis=1).reshape(n,1)),axis=0)/len(Y)
        if E.all() <= threshold: # E_k小于阈值，停止迭代 
            parameter=v_cur,gamma_cur,w_cur,theta_cur
            return BP_result(predict=Y_hat,parameter=parameter)
        g = Y_hat*(1 - Y_hat)*(Y - Y_hat)
        e = b*(1 - b)*np.dot(g,w_cur)
        v_cur += eta*np.dot(e.T,X)
        gamma_cur += -eta*e
        w_cur += eta*np.dot(g.T,b)
        theta_cur += -eta*g
    parameter=v_cur,gamma_cur,w_cur,theta_cur
    return BP_result(predict=Y_hat,parameter=parameter)

In [4]:
result_BP = BP(X=X,Y=Y,q=15)
predict_BP = [1 if each[0]>=each[1] else 0 for each in result_BP.predict]
accuracy = np.sum([data0['好瓜'].values.flatten() == predict_BP])/len(data0)
print(f'准确率：{accuracy}')
predict_BP
result_BP.predict

准确率：1.0


[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

array([[0.99145137, 0.00821363],
       [0.96074113, 0.04116477],
       [0.98380105, 0.01569879],
       [0.97273749, 0.02855297],
       [0.93940804, 0.05919846],
       [0.9392943 , 0.06197716],
       [0.87144086, 0.13409763],
       [0.86994431, 0.1290403 ],
       [0.07498202, 0.92826891],
       [0.05330655, 0.94734122],
       [0.02074829, 0.97983439],
       [0.02110083, 0.97904266],
       [0.05042665, 0.94615366],
       [0.03829997, 0.96033018],
       [0.14834663, 0.84762768],
       [0.00281113, 0.99712972],
       [0.02858429, 0.97280345]])

In [5]:
result_ABP = ABP(X=X,Y=Y,q=15)
predict_ABP = [1 if each[0]>=each[1] else 0 for each in result_ABP.predict]
accuracy = np.sum([data0['好瓜'].values.flatten() == predict_ABP])/len(data0)
print(f'准确率：{accuracy}')
predict_ABP
result_ABP.predict

准确率：0.5294117647058824


[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]

array([[0.99274753, 0.99294701],
       [0.99518231, 0.99538995],
       [0.99491757, 0.99465035],
       [0.99403788, 0.99372276],
       [0.99243774, 0.991767  ],
       [0.99215616, 0.9922073 ],
       [0.99192101, 0.99202286],
       [0.99296709, 0.99306767],
       [0.00658237, 0.00690458],
       [0.00843318, 0.00889915],
       [0.00359811, 0.00397762],
       [0.003472  , 0.00360604],
       [0.00678775, 0.00696404],
       [0.00691933, 0.00662707],
       [0.00862378, 0.00930973],
       [0.00423697, 0.00289196],
       [0.00570112, 0.00500253]])

## 5.6 试设计一个 BP 改进算法，能通过动态调整学习率显著提升收敛速度。编程实现该算法，并选择两个 UCI 数据集与标准 BP 算法进行实验比较。

日后再补。